# Data Preparation

In [484]:
import os
import re
import pandas as pd
from pandas.api.types import CategoricalDtype

In [485]:
def create_dataframe_from_races_csvs_multi_years(base_path, years):
    required_columns = ['Pos', 'Car #', 'Class', 'Drivers', 'Team', 'Car', 'Time', 'Laps', 'Gap']
    all_data = []  # List to hold all data rows

    for year in years:  # Process each year in the list
        year_path = os.path.join(base_path, str(year))  # Path to the specific year

        if not os.path.exists(year_path):
            print(f"The directory for year {year} does not exist.")
            continue  # Skip to the next year if the directory doesn't exist

        for meeting in os.listdir(year_path):  # Iterate through all meetings in the year
            meeting_path = os.path.join(year_path, meeting)
            races_path = os.path.join(meeting_path, "Races")  # Path to the 'Races' folder

            if os.path.exists(races_path):
                for race_file in os.listdir(races_path):  # Iterate through all race files
                    file_path = os.path.join(races_path, race_file)
                    try:
                        df = pd.read_csv(file_path)
                        if set(required_columns).issubset(df.columns):
                            df['Season'] = year
                            df['Meeting'] = meeting.replace("_", " ")
                            df['Race name'] = race_file.replace(".csv", "").replace("_", " ")
                            all_data.append(df[["Season", "Meeting", "Race name"] + required_columns])
                        else:
                            print(f"Skipping {file_path} due to missing required columns.")
                    except Exception as e:
                        print(f"Error reading {file_path}: {e}")

    # Concatenate all data into a single DataFrame
    final_df = pd.concat(all_data, ignore_index=True) if all_data else pd.DataFrame()
    return final_df


## Load the Data About Races 

In [486]:
base_path = ".\data_csv"  # Update to your path
years = [2021, 2022, 2023]
all_races = create_dataframe_from_races_csvs_multi_years(base_path, years)

## Exploratory Data Analysis (EDA)

**`Disclaimer:`** <br>
`I haven't found any information that explains what data these columns actually contain.`<br>
`Next will be my guesses about the true meaning of the columns.`<br><br>


The **raw** dataset contains information about various races, with the following columns:

1. **Season**: The year the race took place;
2. **Meeting**: The location of the meeting;
3. **Race name**: The name of the race;
4. **Pos**: The finishing position in the race;
5. **Car #**: The car number;
6. **Class**: The racing class in which a team competes in a race;
7. **Drivers**: The names of the drivers;
8. **Team**: The name of the team;
9. **Car**: The make and model of the car;
10. **Time**: Best lap time;
11. **Laps**: The number of laps completed;
12. **Gap**: Represent the time difference between each car and the car ahead of it, with a slight twist: <br>
    * If a car is on the same lap as the leader of the race, the Gap shows the actual time difference. <br>
    * If a car is one or more laps behind, the Gap resets and shows the time difference to the car ahead within the same lap, not the overall race leader.
    * **Gap** also stores negative values, according to my assumptions this means that the participant dropped out of the race


In [487]:
all_races.head()

,Season,Meeting,Race name,Pos,Car #,Class,Drivers,Team,Car,Time,Laps,Gap
0,2021,Barcelona,Main Race,1,88,Pro Cup,"Raffaele Marciello, Felipe Fraga, Jules Gounon",AKKA ASP,Mercedes-AMG GT3,1:47.211,95.0,NaN
1,2021,Barcelona,Main Race,2,54,Pro Cup,"Klaus Bachler, Christian Engelhart, Matteo Cai...",Dinamic Motorsport,Porsche 911 GT3-R (991.II),1:47.148,95.0,2.174
2,2021,Barcelona,Main Race,3,32,Pro Cup,"Dries Vanthoor, Robin Frijns, Charles Weerts",Team WRT,Audi R8 LMS GT3,1:47.612,95.0,4.036
3,2021,Barcelona,Main Race,4,63,Pro Cup,"Mirko Bortolotti, Marco Mapelli, Andrea Caldar...",Orange 1 FFF Racing Team,Lamborghini Huracan GT3 Evo,1:47.027,95.0,9.511
4,2021,Barcelona,Main Race,5,4,Pro Cup,"Maro Engel, Luca Stolz, Nico Bastian",HRT,Mercedes-AMG GT3,1:47.588,95.0,9.984


### Data Types and Missing Values

Data Types should be:

1. **Season**: is a numerical and discrete variable, it should be represented as **integer**;
2. **Meeting**: is categorical and nominal variable, it should be represented as **object**;
3. **Race name**: is categorical and nominal variable, it should be represented as **object**;
4. **Pos**: is categorical and ordinal variable, it should be represented as **integer**;
5. **Car #**: is categorical and discrete variable, it should be represented as **integer**;
6. **Class**: is categorical and ordinal variable, it should be represented as **category**;
7. **Drivers**: is categorical and nominal variable, it should be represented as **object**;
8. **Team**: is categorical and nominal variable, it should be represented as **object**;
9. **Car**: is categorical and nominal variable, it should be represented as **object**;
10. **Time**: is numerical and continuous variable, it should be represented as **timedelta**;
11. **Laps**: is a numerical and discrete variable, it should be represented as **integer**;
12. **Gap**: is numerical and continuous variable, it should be represented as **timedelta**;

In [488]:
all_races.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7606 entries, 0 to 7605
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Season     7606 non-null   int64  
 1   Meeting    7606 non-null   object 
 2   Race name  7606 non-null   object 
 3   Pos        7606 non-null   object 
 4   Car #      7606 non-null   int64  
 5   Class      7606 non-null   object 
 6   Drivers    7606 non-null   object 
 7   Team       7606 non-null   object 
 8   Car        7606 non-null   object 
 9   Time       7467 non-null   object 
 10  Laps       7533 non-null   float64
 11  Gap        7389 non-null   object 
dtypes: float64(1), int64(2), object(9)
memory usage: 713.2+ KB


##### Columns data types that differ from the required data type

4.   Pos        7606 non-null   object, should be int;
6.   Class      7606 non-null   object, should be category;
10.   Time       7467 non-null   object, should be timedelta;
11.  Laps       7533 non-null   float64, should be integer;
12.  Gap        7389 non-null   object, should be timedelta.


##### Missing Values

Columns that contain missing values:

10.   Time       7467 non-null   object; 
11.  Laps       7533 non-null   float64;
12.  Gap        7389 non-null   object.



### Inspecting and fix columns

#### 1. Pos
Column `'Pos'` is object, it's mean that most likely it contains non digit values. Then inspect column and select all not digit values.

In [489]:
all_races[all_races['Pos'].str.contains('\D', regex=True, na=False)]['Pos'].value_counts().reset_index()

,Pos,count
0,NC,286
1,DSQ,1


Next delete all rows that contain 'NC' (Not Classified) and 'DSQ' (Disqualified).

'Not Classified': This is a term used to refer to competitors who do not meet certain criteria for classification in the final results of a race or event.

In [490]:
all_races = all_races.loc[~all_races['Pos'].isin(['NC', 'DSQ'])]
all_races.reset_index(drop=True, inplace=True)

Change 'Pos' datatype from object to int

In [491]:
all_races.loc[:, 'Pos'] = all_races['Pos'].astype('int64')

#### 2. Class
Column `'Class'` is object, but we will change it to category.

In [492]:
all_races['Class'].value_counts().reset_index()

,Class,count
0,Pro Cup,2614
1,Silver Cup,1870
2,Pro-AM Cup,1042
3,Gold Cup,827
4,Bronze Cup,823
5,Pro-Am Cup,72
6,AM Cup,44
7,Bronze,21
8,Invitational,6


Change 'Pro-AM Cup' to 'Pro-Am Cup'. 

And 'AM Cup' change to 'Am Cup' for the sake of uniformity. 

And 'Bronze ' change to 'Bronze Cup' according to racing compliance.

In [493]:
all_races['Class'] = all_races['Class'].replace({'Pro-AM Cup': 'Pro-Am Cup', 'AM Cup': 'Am Cup', 'Bronze': 'Bronze Cup'})

all_races['Class'].value_counts().reset_index()

,Class,count
0,Pro Cup,2614
1,Silver Cup,1870
2,Pro-Am Cup,1114
3,Bronze Cup,844
4,Gold Cup,827
5,Am Cup,44
6,Invitational,6


Determining the order of categories and changing data type.

In [494]:
# Determine the order for the main categories
cat_type = CategoricalDtype(categories=[
    'Invitational', 'Am Cup', 'Bronze Cup', 'Pro-Am Cup', 'Silver Cup', 'Gold Cup', 'Pro Cup'
], ordered=True)
# Change the data type of the 'Class' column
all_races['Class'] = all_races['Class'].astype(cat_type)
# Create new column 
all_races.insert(loc=6, column='Special Class', value=False)
# Indicate that 'Invitational' is a special class
all_races.loc[all_races['Class'] == 'Invitational', 'Special Class'] = True

In [495]:
all_races[all_races['Special Class'] == True]['Class']

8      Invitational
15     Invitational
52     Invitational
58     Invitational
98     Invitational
105    Invitational
Name: Class, dtype: category
Categories (7, object): ['Invitational' < 'Am Cup' < 'Bronze Cup' < 'Pro-Am Cup' < 'Silver Cup' < 'Gold Cup' < 'Pro Cup']

#### 3. Time 
The Time column have missing values, and we need to change data type from object to timedelta. First we need to investigate it.

In [496]:
missing_time_rows = all_races[all_races['Time'].isnull()]
missing_time_rows.head()

,Season,Meeting,Race name,Pos,Car #,Class,Special Class,Drivers,Team,Car,Time,Laps,Gap
633,2021,Monza,Main Race after 1.30 hour,41,25,Pro Cup,False,"Adrien Tambay, Alexandre Cougnaud, Christopher...",Sainteloc Racing,Audi R8 LMS GT3,NaN,2.0,-1:18:14.532
634,2021,Monza,Main Race after 1.30 hour,42,70,Pro-Am Cup,False,"Oliver Millroy, Brendan Iribe",Inception Racing,McLaren 720S GT3,NaN,1.0,-1:29:07.011
675,2021,Monza,Main Race after 2.30 hours,41,25,Pro Cup,False,"Adrien Tambay, Alexandre Cougnaud, Christopher...",Sainteloc Racing,Audi R8 LMS GT3,NaN,2.0,-2:16:56.451
676,2021,Monza,Main Race after 2.30 hours,42,70,Pro-Am Cup,False,"Oliver Millroy, Brendan Iribe",Inception Racing,McLaren 720S GT3,NaN,1.0,-2:27:48.930
2246,2022,Barcelona,Main Race after 1.30 hour,47,19,Pro Cup,False,"Leo Roussel, Giacomo Altoe, Arthur Rougier",Emil Frey Racing,Lamborghini Huracan GT3 Evo,NaN,2.0,-1:27:06.233


It seems to me that these rows can be **deleted** because the "Time" column contains the time of the fastest lap, but if there's no time, it means they did not set the fastest lap. Looking at the number of laps completed, one can notice that these people completed only 1 or 2 laps. The "Gap" for them is negative, which, in my guess, indicates that the participant has dropped out of the race, most likely, they withdrew from the race due to an accident or technical malfunction.

In [497]:
all_races = all_races.dropna(subset=['Time'])

To be able to perform calculations with times, we will create a separate column with timedelta values. And in Time' we save the original values.

In [498]:
time_timedelta_values = pd.to_timedelta('00:' + all_races['Time'].astype(str))

all_races.insert(loc=11, column='Time timedelta', value=time_timedelta_values)

#### 4. Laps

In [499]:
all_races.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7279 entries, 0 to 7318
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype          
---  ------          --------------  -----          
 0   Season          7279 non-null   int64          
 1   Meeting         7279 non-null   object         
 2   Race name       7279 non-null   object         
 3   Pos             7279 non-null   object         
 4   Car #           7279 non-null   int64          
 5   Class           7279 non-null   category       
 6   Special Class   7279 non-null   bool           
 7   Drivers         7279 non-null   object         
 8   Team            7279 non-null   object         
 9   Car             7279 non-null   object         
 10  Time            7279 non-null   object         
 11  Time timedelta  7279 non-null   timedelta64[ns]
 12  Laps            7279 non-null   float64        
 13  Gap             7135 non-null   object         
dtypes: bool(1), category(1), float64(1), int64(2)

change laps dtype from float to int

In [500]:
all_races['Laps'] = all_races['Laps'].astype('int64')

#### 5. Gap 

In [501]:
all_races.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7279 entries, 0 to 7318
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype          
---  ------          --------------  -----          
 0   Season          7279 non-null   int64          
 1   Meeting         7279 non-null   object         
 2   Race name       7279 non-null   object         
 3   Pos             7279 non-null   object         
 4   Car #           7279 non-null   int64          
 5   Class           7279 non-null   category       
 6   Special Class   7279 non-null   bool           
 7   Drivers         7279 non-null   object         
 8   Team            7279 non-null   object         
 9   Car             7279 non-null   object         
 10  Time            7279 non-null   object         
 11  Time timedelta  7279 non-null   timedelta64[ns]
 12  Laps            7279 non-null   int64          
 13  Gap             7135 non-null   object         
dtypes: bool(1), category(1), int64(3), object(8),

#### deal with missing values

In [502]:
null_gap_rows = all_races[pd.isnull(all_races['Gap'])]
null_gap_rows['Pos'].value_counts().reset_index()

,Pos,count
0,1,144


The missing values are found among the race leaders; their absence can be explained by the fact that these participants do not have a lag behind the leader, since they are the leaders.

Next we will fill the missing values to '0.000'.

In [503]:
all_races['Gap'] = all_races['Gap'].fillna('0.000')

"Gap" also contain times with '-' before the digits. My assumption is that '-' indicates about cars that droped of the race for some reasons. In futere we will quantify my assumption.


Check out rows with negative 'Gap's

In [504]:
all_races[all_races['Gap'].str.match(r'^-')].head()

,Season,Meeting,Race name,Pos,Car #,Class,Special Class,Drivers,Team,Car,Time,Time timedelta,Laps,Gap
37,2021,Barcelona,Main Race,38,16,Silver Cup,False,"Kikko Galbiati, Tim Zimmermann, Clemens Schmid",GRT Grasser Racing Team,Lamborghini Huracan GT3 Evo,1:48.194,0 days 00:01:48.194000,89,-10:05.535
38,2021,Barcelona,Main Race,39,71,Pro Cup,False,"Callum Ilott, Alessio Rovera, Antonio Fuoco",Iron Lynx,Ferrari 488 GT3,1:47.704,0 days 00:01:47.704000,87,-15:55.406
39,2021,Barcelona,Main Race,40,66,Pro Cup,False,"Mattia Drudi, Steijn Schothorst, Christopher Mies",Attempto Racing,Audi R8 LMS GT3,1:47.443,0 days 00:01:47.443000,86,-17:25.545
40,2021,Barcelona,Main Race,41,2,Pro-Am Cup,False,"Jim Pla, Olivier Grotz, Florian Scholze",GetSpeed,Mercedes-AMG GT3,1:48.768,0 days 00:01:48.768000,75,-36:45.969
41,2021,Barcelona,Main Race,42,5,Silver Cup,False,"Indy Dontje, Patrick Assenheimer, Hubert Haupt",HRT,Mercedes-AMG GT3,1:48.700,0 days 00:01:48.700000,74,-41:38.337


But now we will Create new Column with name 'Dropped off the Race' with bool data type. True if '-' in 'Gap', and False if doesn't. Choose the rows with negative "Gap"

In [505]:
# Function to determine if a participant has dropped off of the race
def check_dropped_off(value):
    if pd.isnull(value):
        return False
    return bool(re.match(r"^-", value))

all_races['Dropped off the Race'] = all_races['Gap'].apply(check_dropped_off)

delete all '-' in 'Gap' column

In [506]:
all_races['Gap'] = all_races['Gap'].str.replace('-', '', regex=False)
all_races.head()

,Season,Meeting,Race name,Pos,Car #,Class,Special Class,Drivers,Team,Car,Time,Time timedelta,Laps,Gap,Dropped off the Race
0,2021,Barcelona,Main Race,1,88,Pro Cup,False,"Raffaele Marciello, Felipe Fraga, Jules Gounon",AKKA ASP,Mercedes-AMG GT3,1:47.211,0 days 00:01:47.211000,95,0.000,False
1,2021,Barcelona,Main Race,2,54,Pro Cup,False,"Klaus Bachler, Christian Engelhart, Matteo Cai...",Dinamic Motorsport,Porsche 911 GT3-R (991.II),1:47.148,0 days 00:01:47.148000,95,2.174,False
2,2021,Barcelona,Main Race,3,32,Pro Cup,False,"Dries Vanthoor, Robin Frijns, Charles Weerts",Team WRT,Audi R8 LMS GT3,1:47.612,0 days 00:01:47.612000,95,4.036,False
3,2021,Barcelona,Main Race,4,63,Pro Cup,False,"Mirko Bortolotti, Marco Mapelli, Andrea Caldar...",Orange 1 FFF Racing Team,Lamborghini Huracan GT3 Evo,1:47.027,0 days 00:01:47.027000,95,9.511,False
4,2021,Barcelona,Main Race,5,4,Pro Cup,False,"Maro Engel, Luca Stolz, Nico Bastian",HRT,Mercedes-AMG GT3,1:47.588,0 days 00:01:47.588000,95,9.984,False


Now add new column 'Gap Timedelta' with the same values from 'Gap' but in different data type.

In [507]:
def convert_to_timedelta(value):
    try:
        # Directly return a timedelta of zero for '0.000'
        if value == '0.000':
            return pd.to_timedelta('00:00:00.000')
        
        # Check the number of parts when split by ':'
        parts = value.split(':')
        if len(parts) == 1:  # Only seconds (and possibly milliseconds)
            return pd.to_timedelta(float(value), unit='s')
        elif len(parts) == 2:  # Minutes and seconds
            return pd.to_timedelta('00:' + value)
        elif len(parts) == 3:  # Hours, minutes, and seconds
            return pd.to_timedelta(value)
    except Exception as e:
        print(f"Error converting value: {value}, Error: {e}")
        # Return a default timedelta in case of unexpected format
        return pd.Timedelta(0)
    

# Apply the conversion to each value in 'Gap'
gaps_timedelta_values = all_races['Gap'].astype(str).apply(convert_to_timedelta)

# Insert the new 'Gap Timedelta' column into the DataFrame
all_races.insert(loc=14, column='Gap Timedelta', value=gaps_timedelta_values)

In [508]:
all_races.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7279 entries, 0 to 7318
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype          
---  ------                --------------  -----          
 0   Season                7279 non-null   int64          
 1   Meeting               7279 non-null   object         
 2   Race name             7279 non-null   object         
 3   Pos                   7279 non-null   object         
 4   Car #                 7279 non-null   int64          
 5   Class                 7279 non-null   category       
 6   Special Class         7279 non-null   bool           
 7   Drivers               7279 non-null   object         
 8   Team                  7279 non-null   object         
 9   Car                   7279 non-null   object         
 10  Time                  7279 non-null   object         
 11  Time timedelta        7279 non-null   timedelta64[ns]
 12  Laps                  7279 non-null   int64          
 13  Gap     